In [240]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [249]:
fdp = "https://www.fdpbt.de/fraktion/abgeordnete"
source_fdp = requests.get(fdp).text
soup_fdp = BeautifulSoup(source_fdp, 'html.parser')

cdu = "https://www.cducsu.de/hier-stellt-die-cducsu-bundestagsfraktion-ihre-abgeordneten-vor"
source_cdu = requests.get(cdu).text
soup_cdu = BeautifulSoup(source_cdu, 'html.parser')

spd = "https://www.spdfraktion.de/abgeordnete/alle?wp=19&view=list&old=19"
source_spd = requests.get(spd).text
soup_spd = BeautifulSoup(source_spd, 'html.parser')

In [277]:
all_abg_fdp = soup.find(class_ = 'person-list').find_all(class_ = 'person-item-wrapper')
all_abg_cdu = soup_cdu.find_all(class_ = 'teaser delegates')
all_abg_spd = soup_spd.find_all(class_ = 'views-row')

In [310]:
twitter_list = []
for abg in all_abg_fdp:
    name_field = abg.find(class_ = 'person-name')
    funktion = name_field.find('span').text.strip()
    name = name_field.text.strip('\n').strip().strip(function).strip('\n').strip()
    twitter = abg.find('a', attrs = {'class': 'tw'}, href = True)
    twitter_list.append(
        {
        'partei': "FDP",
        'name': name,
        'twitter': twitter['href'] if twitter is not None else ""
            }
        )
    
for abg in all_abg_cdu:
    twitter = abg.find(class_ = 'twitter')
    twitter_list.append(
        {
        'partei': "CDU/CSU",
        'name': abg.find('h2').find('span').text,
        'twitter': twitter.find('a', href = True)['href'] if twitter is not None else ""
            }
        )
    
for abg in all_abg_spd:
    twitter = abg.find(class_ = 'ico_twitter')
    test_list.append(
    {
    'partei': "SPD",
    'name': abg.find('h3').find('a').get_text(),
    'twitter': twitter['href'] if twitter is not None else ""
        }
    )

In [311]:
twitter_df = pd.DataFrame(twitter_list)

In [312]:
twitter_df['twitter'] = twitter_df['twitter'].apply(lambda x: x.strip('http://twitter.com/'))
twitter_df['twitter'] = twitter_df['twitter'].apply(lambda x: x.strip('https://twitter.com/'))
twitter_df['twitter'] = twitter_df['twitter'].apply(lambda x: x.strip(''))